# Multi-Arm Bandit Algorithm

## References
1. *Practical Statistics for Data Scientists: 50 Essential Concepts* by Peter Bruce Andrew Bruce
1. [Multi-armed bandit](https://en.wikipedia.org/wiki/Multi-armed_bandit) on Wikipedia

## Problem statement
**Multi-arm bandit**
An imaginary slot machine with multiple arms for the customer to choose from, each with different payoffs, here taken to be an analogy for a multitreatment experiment.

Algorithm is popular A/B testing.

Each arm has an unknown probability of win or, in general, expected payoff.
You have only limited number of tries.
Your goal is to maximize the payoff.

## Exploration vs. Exploitation

Exploration: you spend the first several rounds randomly selecting "arms", then pick the best one and use only that one. But you potentially wasted these rounds by not choosing the best earlier.

Exploitation: you try to figure out quickly which is the best arm and use only it. But you may have picked the wrong one. In that case you should have spent more time exploring.

## Bandit algorithms
Hybrid approach. Keep exploring, but give preference the "arm" with the best results at that moment.

### Epsilon-greedy algorithm

1. Generate a random number between 0 and 1. 
1. If the number lies between 0 and epsilon (where epsilon is a number between 0 and 1, typically fairly small), pick the "arm" at random
3. If the number is ≥ epsilon, show whichever offer has had the highest response rate to date.




### Thompson Sampling

Dynamically adjust the probability we pick samples based on the currently available information.

Bayes rule:

$$ p(A|B) = \frac{p(B|A) p(A)}{p(B)} $$

In our case we would like to estimate the probability distribution of "arms" parameters given the data:

We model the multi-arm bandit using a Multinomial distribution (in case of two arms: Binomial): the probability that we have $n_1$ successes for the first "arm", $n_2$ successes for the second "arm", etc. is:

$$ p(n_1 ... n_k) = \frac{n!}{n_1! ... n_k!} \pi_1^{n_1} ... \pi_k^{n_k}$$

Here  $n = \sum_{i=1}^k{n_i} $ and $\sum_{i=1}^k{ \pi_i }= 1$

The prior distribution of the parameters $\pi$ is Dirichlet distribution:

$$ p(\pi) = \frac{1}{B(\alpha)}  \prod_{i=1}^k {\pi_i^{\alpha_i-1}} $$

Since we don't know anything about the multi-armed bandit until we started interacting with it, it makes sense to set all $\alpha$ to the same value, for example, to 1.

Since the Dirichlet distribution is a conjugate prior of the Multinomial distribution, the posterior distribution is also Dirichlet distribution with parameters $\alpha_i = n_i$






So the algorithm is as follows:
1. Recalculate the parameters of the posterior distribution
2. Draw a sample from the posterior distribution
3. Find the max $\pi_i$ and use the "arm" corresponding to this index

The code below demonstrates that even after 10 trials the worse option can outperform the better option. So if we naively base our decision on the first 10 trials, we may not get the best result.

In [1]:
import random
def draw(p):
    return 1 if random.random() <= p else 0

In [14]:
random.seed(6)
p_a = 0.5
p_b = 0.51

In [15]:
for i in range(10):
    print('A:',draw(p_a), 'B:', draw(p_b))    

A: 0 B: 0
A: 1 B: 1
A: 1 B: 0
A: 1 B: 0
A: 1 B: 0
A: 1 B: 0
A: 0 B: 1
A: 0 B: 0
A: 1 B: 0
A: 0 B: 1


We will use the same parameters, but limit ourselves to only 100 attempts. The best we can get is $51, the worst - $50. 
First we will try epsilon-greedy strategy, and will try to see what is the best value of the epsilon.

In [16]:
def epsilon_greedy(p_a, p_b, epsilon, max_attempts):
    a_successes = 0
    b_successes = 0
    
    for _ in range(max_attempts):
        shold_random = n
    
